In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
# MNIST 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.stack([x_train]*3, axis=-1)
x_test = np.stack([x_test]*3, axis=-1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


In [ ]:
# 이미지 크기 조정
x_train = np.array([np.resize(img, (32, 32, 3)) for img in x_train])
x_test = np.array([np.resize(img, (32, 32, 3)) for img in x_test])


In [ ]:
# 사전 학습된 모델 로드
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

In [ ]:
# 모델 커스터마이징
x = Flatten()(base_model.output)
x = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)


In [ ]:
# 모델 컴파일 및 훈련
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=1, validation_data=(x_test, y_test))


1875/1875 [==============================] - 4588s 2s/step - loss: 0.6245 - accuracy: 0.8679 - val_loss: 0.3409 - val_accuracy: 0.9264


In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
print("\nTest loss:", score[0])
print('Test accuracy:', score[1])

# cf
# 313/313 [==============================] - 1s 3ms/step - loss: 0.0627 - accuracy: 0.9804
# [0.06266114115715027, 0.980400025844574]

313/313 [==============================] - 50s 159ms/step - loss: 0.3409 - accuracy: 0.9264

Test loss: 0.3408912420272827
Test accuracy: 0.9264000058174133


In [ ]:
# 새로운 입력데이터(다운로드 받은 이미지)를 통해 예측 수행

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_new_image(img_path):
    # 사이즈 조정 (모델 훈련 인풋에 맞춤)
    img = image.load_img(img_path, target_size=(32, 32))  # Color image, adjust target size
    # numpy 배열로 변환.
    img_tensor = image.img_to_array(img)
    # 정규화(0-1)
    img_tensor = img_tensor.astype('float32') / 255.0
    # Add a batch dimension
    img_tensor = np.expand_dims(img_tensor, axis=0)

    # 모델을 이용, 입력이미지의 숫자 예측
    prediction = model.predict(img_tensor)
    # 예측된(가장 높은 확률을 가진 클래스) 숫자반환
    # digit = np.argmax(prediction)
    digit = prediction
    formatted_prediction = np.round(prediction, decimals=3)

    return formatted_prediction

In [ ]:
#  이미지 파일 경로 ... 4
img_path ='/content/4.PNG'
predicted_digit = predict_new_image(img_path)
print(f'Predicted digit: {predicted_digit}')

1/1 [==============================] - 0s 57ms/step
Predicted digit: [[0.    0.002 0.996 0.    0.    0.    0.    0.001 0.    0.   ]]


In [ ]:
#  이미지 파일 경로 ... 5
img_path ='/content/5.PNG'
predicted_digit = predict_new_image(img_path)
print(f'Predicted digit: {predicted_digit}')

1/1 [==============================] - 0s 76ms/step
Predicted digit: [[0.    0.002 0.995 0.001 0.    0.    0.001 0.    0.    0.   ]]


In [ ]:
#  이미지 파일 경로 ... 1
img_path ='/content/1_1.JPG'
predicted_digit = predict_new_image(img_path)
print(f'Predicted digit: {predicted_digit}')

1/1 [==============================] - 0s 56ms/step
Predicted digit: [[0.    0.003 0.994 0.001 0.    0.    0.001 0.    0.    0.   ]]
